In [1]:
import pandas as pd
songs = pd.read_csv('../data/songs.csv')
streams = pd.read_csv('../data/streams1.csv')

In [7]:
streams.dtypes

user_id         int64
track_id       object
listen_time    object
dtype: object

In [10]:
songs.head(2)

,id,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.1430,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.166,...,-17.235,1,0.0763,0.9240,0.000006,0.101,0.267,77.489,4,acoustic


In [8]:
streams.head(3)

,user_id,track_id,listen_time
0,26213,4dBa8T7oDV9WvGr7kVS4Ez,2024-06-25 17:43:13
1,6937,4osgfFTICMkcGbbigdsa53,2024-06-25 07:26:00
2,21407,2LoQWx41KeqOrSFra089YS,2024-06-25 13:25:26


In [14]:
# Prepare data
streams['listen_date'] = pd.to_datetime(streams['listen_time']).dt.date
merged_data = streams.merge(songs, on='track_id', how='left')

merged_data.head(2)

,user_id,track_id,listen_time,listen_date,id,artists,album_name,track_name,popularity,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,26213,4dBa8T7oDV9WvGr7kVS4Ez,2024-06-25 17:43:13,2024-06-25,91162,Panic! At The Disco,Pop n' Fresh,Don't Let the Light Go Out,0,229517,...,-4.032,0,0.0313,0.00109,0.00230,0.271,0.179,117.637,4,rock
1,6937,4osgfFTICMkcGbbigdsa53,2024-06-25 07:26:00,2024-06-25,103402,Frank Ocean,Novacane,Novacane,81,302346,...,-9.112,1,0.0919,0.05840,0.00126,0.160,0.370,93.510,4,soul


In [2]:
# Prepare data
streams['listen_date'] = pd.to_datetime(streams['listen_time']).dt.date
merged_data = streams.merge(songs, on='track_id', how='left')

# Compute each KPI
# KPI 1: Daily Genre Listen Count
genre_listen_count = merged_data.groupby(['listen_date', 'track_genre']).size().reset_index(name='listen_count')

# KPI 2: Average Listening Duration per Genre per Day
merged_data['duration_seconds'] = merged_data['duration_ms'] / 1000
avg_duration = merged_data.groupby(['listen_date', 'track_genre'])['duration_seconds'].mean().reset_index(name='average_duration')

# KPI 3: Daily Genre Popularity Index
total_listens = merged_data.groupby('listen_date').size().reset_index(name='total_listens')
genre_listen_count = genre_listen_count.merge(total_listens, on='listen_date')
genre_listen_count['popularity_index'] = genre_listen_count['listen_count'] / genre_listen_count['total_listens']

# KPI 4: Most Popular Track per Genre per Day
most_popular_track = merged_data.groupby(['listen_date', 'track_genre', 'track_id']).size().reset_index(name='track_count')
most_popular_track = most_popular_track.sort_values(by=['listen_date', 'track_genre', 'track_count'], ascending=[True, True, False])
most_popular_track = most_popular_track.drop_duplicates(subset=['listen_date', 'track_genre'], keep='first').rename(columns={'track_id': 'most_popular_track_id'})

# Combine all KPIs into one DataFrame
final_kpis = genre_listen_count[['listen_date', 'track_genre', 'listen_count', 'popularity_index']]
final_kpis = final_kpis.merge(avg_duration, on=['listen_date', 'track_genre'])
final_kpis = final_kpis.merge(most_popular_track[['listen_date', 'track_genre', 'most_popular_track_id']], on=['listen_date', 'track_genre'])

In [3]:
final_kpis.head()

,listen_date,track_genre,listen_count,popularity_index,average_duration,most_popular_track_id
0,2024-06-25,acoustic,118,0.010400,215.379034,0WWuB1F1H4dr3Bdoe1vtHs
1,2024-06-25,afrobeat,124,0.010929,248.978960,1KME77F9mu2RQS8vo6JVwa
2,2024-06-25,alt-rock,119,0.010488,236.073731,0GO8y8jQk1PkHzS31d699N
3,2024-06-25,alternative,45,0.003966,202.768200,0YwBrYaPYYc8e18ZYkqhJc
4,2024-06-25,ambient,119,0.010488,255.296067,2tr4oclswJ6v3dfDlI01HD
